## 네이버 뉴스 크롤링

In [ ]:
# !pip install newspaper3k

In [1]:
import os
import sys
import json
import pandas as pd
from newspaper import Article
from bs4 import BeautifulSoup
import time
import requests
import re

In [8]:
def get_article(url, language='ko'):
    """
    newspaper3k 라이브러리 사용해 기사 제목, 본문 파싱
    """
    a = Article(url, language='ko')
    a.download()
    a.parse()

    return a.title, a.text


# #크롤링할 url 주소 입력
# url = 'http://v.media.daum.net/v/20170604205121164'
# title, context = get_article(url)

In [3]:
def get_ranking_news(date, cnt):
    """
    해당 날짜의 언론사별 조회수 기준 상위 cnt개 기사 정보 수집
    """
    total_time = 0
    I = []
    
    #언론사
    press_id = {"MBC":"214", "KBS":"056", "SBS":"055", "중앙일보":"025", "JTBC":"437", "YTN":"052", "서울신문":"081", "매일경제":"009", "한국경제":"015", "조선일보":"023", "세계일보":"022", "한겨례":"028", "한국일보":"469", "연합뉴스":"001", "채널A":"449", "MBN":"057", "조선비즈":"366", "뉴스1":"421", "미디어오늘":"006"}
    
    for press in press_id:
        start = time.time()
        url = "https://news.naver.com/main/ranking/office.nhn?officeId="+press_id[press]+"&date="+str(date)
        headers = {"User-Agent": "Mozilla/5.0(Windows NT 10.0;Win64;x64)AppleWebKit/537.36(KHTML, like Gecko)Chrome/87.0.4280.88 Safari/537.36"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        
        ranking_box = soup.find_all(class_="rankingnews_box_inner")
        
        
        #조회수(0) -> 댓글수(1)
        for ranking_type in range(1):
            ranking = ranking_box[ranking_type].find_all(class_="list_ranking_num")
            url_list = ranking_box[ranking_type].find_all(class_="list_content")
            
            # 랭킹순
            for rank in range(cnt):
                d = {}
                d['Date'] = int(date)
                d['Press'] = press
                d['Rank'] = ranking[rank].get_text()
                d['URL'] = url_list[rank].find('a')['href']
                d['Title'] = url_list[rank].find('a').get_text()
                if(ranking_type == 0):
                    d['View'] = url_list[rank].find(class_="list_view").get_text()
                elif(ranking_type==1):
                    d['Comment'] = url_list[rank].find(class_="list_comment nclicks('RBP.dcmtnwscmt')").get_text()
                I.append(d)
    return I

In [7]:
date = "20230521"
cnt = 1
save_path = './news_data/' + str(date) +'_ranking_news.xlsx'

ranking_news = get_ranking_news(date, cnt)
total_df = pd.DataFrame(columns=["title", "text", "url", "date", "press"])

for news in ranking_news:
    url = "https://news.naver.com" + news['URL']
    title, text = get_article(url, language='ko')

    news_data = pd.DataFrame({
        "title": [title],
        "text": [text],
        "url": [url],
        "date": [news["Date"]],
        "press": [news["Press"]]
    })

    total_df = pd.concat([total_df, news_data])

total_df.to_excel(save_path, index=False)
print('Saved to...', save_path)

Saved to... ./news_data/20230521_ranking_news.xlsx


In [5]:
total_df.head()

,title,text,url,date,press
0,서울대 경제ㆍ경영 졸업생들은 어디로 가나?,"""행시(행정고시)도 시들해졌어요"" ""유학도 많이 줄었어요""\n\n'신의 직장'으로 ...",https://news.naver.com/main/ranking/read.naver...,20230521,MBC
0,인천 서구·경기 김포 합의…5호선 김포-검단 연장 급물살,"\n\n■ 제보하기\n\n▷ 전화 : 02-781-1234, 4444\n\n▷ 이메...",https://news.naver.com/main/ranking/read.naver...,20230521,KBS
0,"[단독] 맨발 여중생 폭행한 가족…""아빠는 구치소에 구금""",<앵커>\n\n\n\n서울 강남 한복판에서 여중생이 부모와 오빠에게 집단 폭행을 당...,https://news.naver.com/main/ranking/read.naver...,20230521,SBS
0,"'100억 건물주' 하지원…1층 안경점 빼고 텅 비었다, 年2억 손실?",배우 하지원이 서울 중심에 있는 빌딩을 100억원에 매입했으나 임대 수익률이 낮아 ...,https://news.naver.com/main/ranking/read.naver...,20230521,중앙일보
0,[영상] 제트스키 몰고 남방큰돌고래 코앞에서 위협한 6명 적발,제주 바다에서 제트스키를 몰고 해양보호생물인 남방큰돌고래 무리에 과도하게 가까이 다...,https://news.naver.com/main/ranking/read.naver...,20230521,JTBC
